In [1]:
import hail as hl

hl.init(
    tmp_dir='/net/ascratch/people/plggosborcz/gosborcz-hail',
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38') 

2023-03-29 20:12:41.335 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0746:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230329-2012-0.2.109-b71b065e4bb6.log


In [ ]:
# run PCA
# first prepare a matrix for PCA by LD pruning and MAF > 0.05 filter
# consider WGS_8 and WGS_5 to delete - do this after keeping only founders of the families

for_pca = mt.filter_rows(mt.variant_qc.AF[1] > 0.05)
biallelic_dataset = for_pca.filter_rows(hl.len(for_pca.alleles) == 2)
pruned_variant_table = hl.ld_prune(biallelic_dataset.GT, r2=0.2, bp_window_size=500000)
for_pca = for_pca.filter_rows(hl.is_defined(pruned_variant_table[for_pca.row_key]))

for_pca = for_pca.checkpoint(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/to-delete-for-pca.mt'
)

# after initial check only keep one founder sample per family and skip the rest
eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT)
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.group,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p) #these for now are far away from each other but for SKAT it may be different,

#split multiallelic variants
#annotate per-gene (with interval)

# add synthetic controls from the Polish database
#run SKAT

In [9]:
genes = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2', 'NR2F1', 'ASH1L', 'FLT3']
#list of GTS genes updated with 3 recently discovered genes

In [16]:
var_s = hl.import_table('supplementary-table-S2.csv', delimiter = ',', quote = "\"")

2023-03-29 20:30:06.208 Hail: INFO: wrote table with 99 rows in 1 partition to /net/ascratch/people/plggosborcz/gosborcz-hail/persist_tablejdWxuLRkas
2023-03-29 20:30:06.665 Hail: INFO: Reading table without type imputation
  Loading field '' as type str (not specified)
  Loading field 'locus.contig' as type str (not specified)
  Loading field 'locus.position' as type str (not specified)
  Loading field 'alleles' as type str (not specified)
  Loading field 'gnomad_v3.v3_nfe.AC' as type str (not specified)
  Loading field 'gnomad_v3.v3_nfe.AF' as type str (not specified)
  Loading field 'gnomad_v3.v3_nfe.AN' as type str (not specified)
  Loading field 'gnomad_v3.v3_nfe.homozygote_count' as type str (not specified)
  Loading field 'gnomad_v3.most_severe_consequence' as type str (not specified)
  Loading field 'gnomad_v3.regulatory_feature_consequences' as type str (not specified)
  Loading field 'gnomad_v3.transcript_consequences' as type str (not specified)
  Loading field 'gnomad_v3.rs

In [19]:
var_s = var_s.transmute(
    locus = hl.locus(
        var_s['locus.contig'],
        hl.int(var_s['locus.position']),
        reference_genome='GRCh38')
)

In [21]:
var_s = var_s.key_by(var_s.locus)

In [22]:
var_s.write('/net/ascratch/people/plggosborcz/var_s.ht')

2023-03-29 20:33:45.014 Hail: INFO: Coerced sorted dataset
2023-03-29 20:33:45.812 Hail: INFO: wrote table with 98 rows in 1 partition to /net/ascratch/people/plggosborcz/var_s.ht


In [ ]:
var_s = hl.read_table('/net/ascratch/people/plggosborcz/var_s.ht')
mt = mt.key_rows_by(mt.locus)
mt = mt.filter_rows(hl.is_defined(var_s[mt.row_key]))
mt = mt.key_rows_by(mt.locus, mt.alleles)

# for preliminary PCA chose only one member of each family
mt = mt.filter_cols(
    hl.if_else(
        mt.group == 'GTS',
        ((mt.phenotypes.kinship == 'P') | (mt.phenotypes.kinship == 'proband')),
        True
    )
)